In [6]:
import numpy as np

In [108]:
path = "../src_v7_sent_prmpt_ele-wise-mul/res/renew_lr1e-5_ep5_add_bs96_kl02_tk10_tp07_tm1o1_rpt1o5_1.txt"
with open(path, "r") as f: # samples_cl_ln_lr1e-5_ep3
    data1 = f.readlines()
data1 = [item.strip() for item in data1]
# with open("./baselines/gpt2_samples_tk10_tp07_tm1o1_rpt1o5_ep3.txt", "r") as f: # samples_cl_ln_lr1e-5_ep3
#     data1 = f.readlines()
# data1 = [item.strip() for item in data1]
with open("../src_v4_new_gpt/res/samples_wocl_ln_lr1e-5_ep3.txt", "r") as f:
    data2 = f.readlines()
data2 = [item.strip() for item in data2]
with open("../src_wo_img/res/samples_wo_img_lr1e-5_ep3.txt", "r") as f:
    data3 = f.readlines()
data3 = [item.strip() for item in data3]
with open("../src_wo_text/res/samples_wo_text_lr1e-5_ep3.txt", "r") as f:
    data4 = f.readlines()
data4 = [item.strip() for item in data4]



with open("baselines/plan_samples.txt", "r") as f:
    data5 = f.readlines()
data5 = [item.strip() for item in data5]
with open("baselines/wm_samples.txt", "r") as f:
    data6 = f.readlines()
data6 = [item.strip() for item in data6]
with open("baselines/gpt2_samples.txt", "r") as f:
    data7 = f.readlines()
data7 = [item.strip() for item in data7]
with open("baselines/iprompt_samples.txt", "r") as f:
    data8 = f.readlines()
data8 = [item.strip() for item in data8]
with open("baselines/iprompt_labels.txt", "r") as f:
    labels8 = f.readlines()
labels8 = [item.strip() for item in labels8]
with open("baselines/img2poem_samples.txt", "r") as f:
    data9 = f.readlines()
data9 = [item.strip() for item in data9]

## 计算diversity

In [73]:
# def create_ngram_set(input_list, ngram_num):
#     if len(input_list) < ngram_num:
#         return set()
#     if len(input_list) <= ngram_num:
#         # print('111:', input_list, {tuple(list(input_list))})
#         return {tuple(list(input_list))}
#     else:
#         # print('222:', input_list, set(zip(*[input_list[i:] for i in range(ngram_num)])))
#         return set(zip(*[input_list[i:] for i in range(ngram_num)]))

In [74]:
# a = create_ngram_set("这是测试文本",2)
# b = create_ngram_set("另一个文本", 2)
# print(a, b, a|b, a&b, a^b, len(a&b)/len(a|b))

In [22]:
# def calculate_1_diversity(samples, ngram_n):
#     res = []
#     for poem in samples:
#         tmp = set()
#         lines = poem.split('，')
#         for line in lines:
#             tmp |= create_ngram_set(line, ngram_n)
#         res.append(tmp)
# #     print(res[0], res[1])
#     numerator = res[0] ^ res[1] ^ res[2] ^ res[3] ^ res[4] ^ res[5] ^ res[6] ^ res[7] ^ res[8] ^ res[9]
#     denominator = res[0] | res[1] | res[2] | res[3] | res[4] | res[5] | res[6] | res[7] | res[8] | res[9] 
#     # print(len(numerator), len(denominator))
#     return len(numerator) / len(denominator)

In [23]:
# def calculate_N_diversity(inputs, ngram):
#     res = []
#     for i in range(0, len(inputs), 10):
#         div = calculate_1_diversity(inputs[i:i+10], ngram)
#         res.append(div)
#     # print(res)
#     return np.mean(res)

In [24]:
# ngram = 1
# res1 = calculate_N_diversity(data1, ngram)
# res2 = calculate_N_diversity(data2, ngram)
# res3 = calculate_N_diversity(data3, ngram)
# res4 = calculate_N_diversity(data4, ngram)
# res5 = calculate_N_diversity(data5, ngram)
# res1, res2, res3, res4, res5

## 计算distinct

In [2]:
len(data1), data1[0]

(500,
 '不要说我们的爱情，只是一个梦，你对他没有感觉，但却还在心里，想着曾经拥抱，那种温暖，如今变成了寂寞，我知道你也很难过，因为太多伤害，所以才让自己')

In [4]:
from paddlenlp.metrics import Distinct
distinct = Distinct(n_size=2)
cand = ["The","cat","The","cat","on","the"]
distinct.add_inst(cand)
print(distinct.score()) # 0.8333333333333334

0.8


In [109]:
def cal_dist(data, n):
    res = []
    for i in range(0,len(data),10):
        lines = data[i:i+10]
        distinct = Distinct(n_size=n)
        for line in lines:
            distinct.add_inst(list(line))
        res.append(distinct.score())
#     print(distinct.score())
    print(np.mean(res))
# n = 1
# cal_dist(data1, n), cal_dist(data2, n), cal_dist(data3, n), cal_dist(data4, n), \
# cal_dist(data5, n), cal_dist(data6, n), cal_dist(data7, n)
cal_dist(data1, 1), cal_dist(data1, 2), cal_dist(data1, 3)

0.32429162899197544
0.7426630701845105
0.919122566062851


(None, None, None)

### 计算img2poem的distinct

In [102]:
def cal_dist(data, n):
    res = []
    i = 0
    distinct = Distinct(n_size=n)
    while i < len(data):
        line = data[i]
        distinct.add_inst(list(line))
        res.append(distinct.score())
        i += 1
#     print(distinct.score())
    print(np.mean(res))
cal_dist(data9, 1), cal_dist(data9, 2), cal_dist(data9, 3)

0.20010713952852385
0.41260806296058716
0.5301637227731582


(None, None, None)

### 计算iprompt的distinct

In [11]:
labels8[:5], data8[:5]

(['曲编雷颂德词黄伟文，听旁人说很满足，却从未揣摩清楚，人们实在说什么，在绝地种出花朵，从未试过叫我满面春风过，你来提我好与多，太容易分不清楚，问我实在欠什么，像大漠变出运河',
  '曲编雷颂德词黄伟文，听旁人说很满足，却从未揣摩清楚，人们实在说什么，在绝地种出花朵，从未试过叫我满面春风过，你来提我好与多，太容易分不清楚，问我实在欠什么，像大漠变出运河',
  '曲编雷颂德词黄伟文，听旁人说很满足，却从未揣摩清楚，人们实在说什么，在绝地种出花朵，从未试过叫我满面春风过，你来提我好与多，太容易分不清楚，问我实在欠什么，像大漠变出运河',
  '曲编雷颂德词黄伟文，听旁人说很满足，却从未揣摩清楚，人们实在说什么，在绝地种出花朵，从未试过叫我满面春风过，你来提我好与多，太容易分不清楚，问我实在欠什么，像大漠变出运河',
  '始终相守相勉，誓约山盟今生永存，为你从今不讲再见，长伴裙边永共缠绵，愿你同相依，每天朝晚得见，莫再分离相思寸寸，渡过困苦情义倍添，要共寻理想心更坚，就算理想如梦似烟'],
 ['能将多少难割舍?才讲慷慨为君子，分付人生牢牢记,怎定情景处处齐，一双通臂勇发劲,仗剑还须君不疑，有心活泼多欢悦,一去掀开解决时',
  '能将多少难割舍?登高驶目我心畅，依稀旧游同为客,零落回忆多感伤，错剪传说滋味远,离别伤痛渐觉凉，却嫌孑然平生过,还是相交叙旧强',
  '一壶浊酒浇恨意,而今遗失步无鞋，爱人怒我登墙去,几度轻如落叶飞，纵眼望去长路渺,不经鸿蒙怎堪期，日后慎保怀自任,离别岂能惧相思',
  '造化弄人终偶然,为人求索反一生，百转千回多是惑,浮生一线是良时',
  '除却苍穹日月星,涓流永长向东游，爱人不见既姓名,无处可寻拉忧愁，成双成对昼夜沉,一曲清泉流幽柔，环抱一问尽此生,伴君消受又惆怅'])

In [15]:
def cal_dist(data, n):
    res = []
    i = 0
    while i < len(data):
        nsamples = labels8.count(labels8[i])
        lines = data[i:i+nsamples]
        distinct = Distinct(n_size=n)
        for line in lines:
            distinct.add_inst(list(line))
        res.append(distinct.score())
        i += nsamples
#         break
#     print(distinct.score())
    print(np.mean(res))
cal_dist(data8, 1), cal_dist(data8, 2), cal_dist(data8, 3)

0.7280950311214829
0.9490035091288873
0.9585102090785966


(None, None, None)

In [17]:
def calculate_1_distinct(samples, ngram_n):
    res = []
#     for poem in samples:
#         distinct = Distinct(n_size=ngram_n)
#         lines = poem.split(':')[1].split('|')
#         for line in lines:
#             distinct.add_inst(list(line))
#         res.append(distinct.score())

    poems = []
    for poem in samples:
        lines = poem.split('，')
        poems.append(lines)
#     print(len(poems[0]), len(poems[1]), len(poems[2]), len(poems[3]))
    for i in range(len(poems[0])):
        try:
            distinct = Distinct(n_size=ngram_n)
            for s in range(len(poems)):
                line = poems[s][i]
                distinct.add_inst(list(line))
            res.append(distinct.score())
        except:
            continue
    # print(res)
    return np.mean(res)

In [18]:
def calculate_N_distinct(inputs, ngram): # 240
    res = []
    for i in range(0, len(inputs), 10):
        div = calculate_1_distinct(inputs[i:i+10], ngram)
        res.append(div)
    # print(res)
    return np.mean(res)

In [110]:
ngram = 2
res1 = calculate_N_distinct(data1, ngram)
res2 = calculate_N_distinct(data2, ngram)
res3 = calculate_N_distinct(data3, ngram)
res4 = calculate_N_distinct(data4, ngram)
res1, res2, res3, res4

(0.9154657743260896, 0.911991648948531, 0.9054952725989203, 0.9114804788696582)

In [67]:
from paddlenlp.metrics import Distinct
distinct = Distinct(n_size=2)
cand = ["The","cat","The","cat","on","the"]
distinct.add_inst(cand)
print(distinct.score()) # 0.8333333333333334

0.8


In [25]:
cand = ["The1","cat1","on1","the1","mat1"]
distinct.add_inst(cand)
print(distinct.score()) # 0.8333333333333334

0.8333333333333334


In [133]:
c = {"apple", "banana", "cherry"}
d = {"apple1", "banana", "cherry"}
e = {"apple1", "banana", "cherry1"}
f = {"apple1", "banana", "cherry1"}
print(c^d) # {'apple', 'apple1'} ?
print(c^d^e) # {'apple', 'cherry1'} ?
print(c^d^e^f) # {'apple'} ?
print((c-d)&(c-e)|(c-f)|(d-e)|(e-f))

{'apple1', 'apple'}
{'cherry1', 'banana', 'apple'}
{'apple', 'apple1'}
{'apple', 'cherry'}
